In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Dense, Input


weights = np.array([87,81,82,92,90,61,86,66,69,69,76,53,70])
heights = np.array([187,174,179,192,188,160,179,168,168,174,182,162,176])

weight_scaler = StandardScaler()
height_scaler = StandardScaler()
weights_scaled = weight_scaler.fit_transform(weights.reshape(-1,1))
heights_scaled = height_scaler.fit_transform(heights.reshape(-1,1))


# 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(weights_scaled, heights_scaled, test_size=0.2, random_state=42)

# 모델 1: 단일 레이어 , Sequentail 모델 초기화
model = Sequential([
    Input(shape=(1,)),  # Input 객체를 사용
    Dense(units=1)      # 이후 레이어 정의
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Param 이 두개라는 건, 가중치 하나, 바이어스 하나. 총 두개.라는 얘기고

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2 (8.00 B)

 Trainable params: 2 (8.00 B)

 Non-trainable params: 0 (0.00 B)

verbose=0은 fit 메서드에서 사용되고 모델 학습 중 출력되는 로그의 상세 정도를 설정하는 옵션

1. verbose=0:
- 아무런 로그를 출력하지 않습니다.
- 학습 상태를 확인하지 않아도 될 때 유용합니다.

1. verbose=1 (기본값):
- 학습 진행 상황을 한 줄씩 출력합니다.
- 에포크별로 손실 값을 표시하며, 진행 바(progress bar)가 표시됩니다.

In [2]:
model.fit(X_train, y_train, epochs=100, verbose=0)

In [3]:
## 모델2 Hidden Layer를 포함한 아키텍쳐
"""

model2 = Sequential()
model2.add(Dense(units= 64, activation='relu',input_shape = [1]))
model2.add(Dense(units= 64, activation='relu'))
model2.add(Dense(units=1 ))
"""


model2 = Sequential([
    Input(shape=(1,)),
    Dense(units=64, activation='relu'),
    Dense(units=64, activation='relu'),
    Dense(units=1)
])


model2.compile(optimizer= 'adam',loss= 'mean_squared_error')
model2.summary()
model2.fit(X_train, y_train, epochs=150, verbose=0)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                 │ (None, 64)             │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,353 (17.00 KB)

 Trainable params: 4,353 (17.00 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
#예측 및 평가
predictions = model.predict(X_test)
predictions2 = model2.predict(X_test)

mse_model1 = mean_squared_error(y_test, predictions)
mse_model2 = mean_squared_error(y_test, predictions2)

print(f"Model 1 MSE: {mse_model1}")
print(f"Model 2 MSE: {mse_model2}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Model 1 MSE: 0.24389709830471898
Model 2 MSE: 0.44258656811198294


스케일링 후 Model 2의 MSE가 낮아진 것은 긍정적인 결과다.</br>
은닉층이 있는 복잡한 모델에서는 스케일링된 데이터를 사용하면 좋은 성능을 얻을 가능성이 크다

In [ ]:
def predict_height(weight):
    # 입력 데이터를 2D 배열로 변환
    weight_array = np.array([[weight]])
    # 스케일링 적용
    weight_scaled = weight_scaler.transform(weight_array)
    # 예측
    height_scaled = model2.predict(weight_scaled)
    # 스케일 복원
    height_original = height_scaler.inverse_transform(height_scaled)
    
    # 결과 출력
    return f"predict({weight}) = {height_original[0, 0]:.2f}"
    
# 예측
result = predict_height(90)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
predict(90) = 69.91
